In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import datetime
import imutils
import time
import dlib
import cv2

In [2]:
#Using dlib's pre trained face and shape detector to localise face in the image and then other facial features
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/amit/driver1/facial-landmarks/shape_predictor_68_face_landmarks.dat')
#In the above line, use the address at which you saved the shape_predictor file

In [ ]:
#Starting the webcam live-video stream
vs = VideoStream(0).start()
time.sleep(2.0)

while True:

    frame = vs.read()
    #frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    #Extracting left and right eye images from face
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for (name, (j, k)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            (x, y, w, h) = cv2.boundingRect(np.array([shape[j:k]]))
            roi = gray[y:y + h, x:x + w]
            #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            if(j==36):
                right_eye = roi
            elif(j==42):
                left_eye = roi
                
    cv2.imwrite('right_eye.jpg', right_eye)
    right_eye = cv2.imread('right_eye.jpg',0)
        
    #pupil detection using cdf
    
    img_flat = right_eye.flatten()
    
    #In below line np.histogram() returns the frequency of each intensity value(from 0 to 255) according to the image
    #Then we calculate their probabilities by dividing each frequency value with total image size and storing it in
    #pixel_prob
    hist, bin_edges = np.histogram(img_flat, 256, [0,256])  
    pixel_prob = hist/img_flat.size
    
    #In pixel_cmprob, we are storing the cumulative probability of each intensity value
    pixel_cmprob = np.zeros((256,1))
    pixel_cmprob[0] = pixel_prob[0]

    for i in range (1,256):
        pixel_cmprob[i] = pixel_cmprob[i-1] + pixel_prob[i]
    
    #Here we loop over each pixel in the image. If the intensity value in the pixel has less value than 0.06 in pixel_cmprob
    #then we change the pixel value to 255 otherwise to zero.
    new_img = np.zeros((right_eye.shape[0],right_eye.shape[1]))
    for i in range(0, right_eye.shape[0]):
        for j in range(0, right_eye.shape[1]):
            if pixel_cmprob[right_eye[i,j]] < 0.06:
                new_img[i,j] = 255
            else:
                new_img[i,j] = 0
    #We erode the image to remove small disturbances using 2*2 window                    
    kernel = np.ones((2,2), np.uint8)
    new_img_er1 = cv2.erode(new_img, kernel, iterations=1)
    
    #Here we calculate the PMI(pixel with minimum intenstiy) of those pixels in the original right_eye image
    #whose corresponding value in new_img_er1 is 255.
    min_arr = np.array([0,0])
    for i in range(0, right_eye.shape[0]):
        for j in range(0, right_eye.shape[1]):
            if(new_img_er1[i,j]==255):
                if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                        min_arr[0] = i
                        min_arr[1] = j
                        
    #We calculate the first threshold value as the average of intensities of 10*10 window around the PMI    
    thresh_val1 = np.mean(right_eye[(min_arr[0]-5):(min_arr[0]+5), (min_arr[1]-5):(min_arr[1]+5)])
        
    right_eye_f1 = right_eye
    _,right_eye_f1 = cv2.threshold(right_eye_f1, thresh_val1, 255, cv2.THRESH_BINARY_INV)
    
    #We again calculate the PMI, now using right_eye_f1 instead of new_img_er1 as before
    min_arr = np.array([0,0])
    for i in range(0, right_eye.shape[0]):
        for j in range(0, right_eye.shape[1]):
            if(right_eye_f1[i,j]==255):
                if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                        min_arr[0] = i
                        min_arr[1] = j
    
    #We calculate the second threshold value as the average of intensities of 15*15 window around the PMI
    thresh_val2 = np.mean(right_eye[(min_arr[0]-7):(min_arr[0]+7), (min_arr[1]-7):(min_arr[1]+7)])
        
    right_eye_f2 = right_eye
    _,right_eye_f2 = cv2.threshold(right_eye_f2, thresh_val2, 255, cv2.THRESH_BINARY_INV)
    
    #We again calculate PMI using right_eye_f2. This will be our assumed Pupil centre.
    min_arr = np.array([0,0])
    for i in range(0, right_eye.shape[0]):
        for j in range(0, right_eye.shape[1]):
            if(right_eye_f2[i,j]==255):
                if(right_eye[i,j]<right_eye[min_arr[0], min_arr[1]]):
                        min_arr[0] = i
                        min_arr[1] = j
        
    right_eye2 = right_eye
    #Drawing a circle with the assumed pupil centre as centre
    cv2.circle(right_eye2, (min_arr[1],min_arr[0]), 5, (0,0,255), -1)
    cv2.imshow("right_eye2", right_eye2)
                
    key = cv2.waitKey(1) & 0xFF
if (key == ord("q")):
    break  
        
        
cv2.destroyAllWindows()
vs.stop()

/home/amit/driver1/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/amit/driver1/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
